Import modules

In [1]:
# public
import os
import argparse
import random
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torch.autograd import Variable

import torchvision
import torchvision.datasets as datasets
import torchvision.transforms as transforms

# custom
from DCL_finegrained import model
from utee import selector
from utee.Normalize import Normalize
from loader_checkpoint import *

# logging
# import logging, json
# with open("logging_config.json", "rt") as file:
#     config = json.load(file)
# logging.config.dictConfig(config)
# logger = logging.getLogger()

import ssl
ssl._create_default_https_context = ssl._create_unverified_context


ArgumentParser

In [2]:
parser = argparse.ArgumentParser(description='Transferable Perturbation via Frequency Manipulation')
parser.add_argument('--epochs', type=int, default=0, help='Model checkpoint epoch number')
parser.add_argument('--eps', type=int, default=10, help='Perturbation budget (0~255)')
parser.add_argument('--model_type', type=str, default='vgg16', help='Victim model: vgg16, vgg19, res152, dense169')
parser.add_argument('--RN', type=lambda x: (str(x).lower() == 'true'), default=False, help='If true, activating the Random Normalization module in training phase')
parser.add_argument('--DA', type=lambda x: (str(x).lower() == 'true'), default=False, help='If true, activating the Domain-agnostic Attention module in training phase')
parser.add_argument('--FA', type=lambda x: (str(x).lower() == 'true'), default=False, help='If true, activating the Frequency Augmentation module in training phase')
parser.add_argument('--iter_ckpt', type=lambda x: (str(x).lower() == 'true'), default=False, help='If true, Model checkpoint with iteration number')
parser.add_argument('--iter', type=int, default=10000, help='Model checkpoint iteration number')
args = parser.parse_args(args=[])
print(args)

Namespace(DA=False, FA=False, RN=False, epochs=0, eps=10, iter=10000, iter_ckpt=False, model_type='vgg16')


Eval

In [3]:
# Choose the domain sets for evaluating the cross-domain transferability
domain_list = ['cifar10', 'cifar100', 'stl10', 'svhn', 'dcl_dub', 'dcl_car', 'dcl_air', 'imagenet', 'imagenet_incv3']
domain_selected = domain_list[:4]
# domain_selected = domain_list[-2:-1] # imagenet
# domain_selected = domain_list[-3:-2] # dcl_air
domain_selected = domain_list[-4:-3] # dcl_car
print(domain_selected)

['dcl_car']


In [4]:
for domain in domain_selected:
    print('='*30, '{}'.format(domain), "="*30)
    
    # Load the victim model (imagenet)
    if domain[:3] == 'dcl': # CUB-200-2011, Stanford Cars, FGVC Aircraft
        batch_size = 6
        if domain == 'dcl_cub':
            numcls = 200
        elif domain == 'dcl_car':
            numcls = 196
        elif domain == 'dlc_air':
            numcls = 100
    elif domain == 'imagenet_incv3':
        batch_size = 16
    elif domain == 'imagenet':
        batch_size = 32
        mean, std = [0.485, 0.456, 0.406], [0.229, 0.224, 0.225]
        model_vgg16 = nn.Sequential(Normalize(mean,std), torchvision.models.vgg16(pretrained=True)).cuda().eval()
        model_vgg19 = nn.Sequential(Normalize(mean,std), torchvision.models.vgg19(pretrained=True)).cuda().eval()
        model_res50 = nn.Sequential(Normalize(mean,std), torchvision.models.resnet50(pretrained=True)).cuda().eval()
        model_res152 = nn.Sequential(Normalize(mean,std), torchvision.models.resnet152(pretrained=True)).cuda().eval()
        model_dense121 = nn.Sequential(Normalize(mean,std), torchvision.models.densenet121(pretrained=True)).cuda().eval()
        model_dense169 = nn.Sequential(Normalize(mean,std), torchvision.models.densenet169(pretrained=True)).cuda().eval()
    else: # CIFAR-10, CIFAR-100, STL-10, SVHN
        batch_size = 128
    
    # Load the validation dataset & victim model (others)
    if domain == 'imagenet':
        ds_fetcher, is_imagenet = selector.select(domain)
    elif domain[:3] == 'dcl': # CUB-200-2011, Stanford Cars, FGVC Aircraft
        model_res50, model_senet, model_seres101, ds_fetcher, is_imagenet = selector.select(domain)
        acc_res50, clean_res50, acc_senet, clean_senet, acc_seres101, clean_seres101 = 0,0,0,0,0,0
    else:  # CIFAR-10, CIFAR-100, STL-10, SVHN
        model_raw, ds_fetcher, is_imagenet = selector.select(domain)
        
    if domain[-5:] == 'incv3':
        ds_val = ds_fetcher(batch_size=batch_size, input_size=299, train=False, val=True)
        data_length = len(ds_fetcher(batch_size=1, train=False, val=True))
    else:
        ds_val = ds_fetcher(batch_size=batch_size, train=False, val=True)
        data_length = len(ds_fetcher(batch_size=1, train=False, val=True))
    print('Validation data length:', data_length)
    
    
    # Load the generative model (attacker)
    netG = load_gan(args, domain).cuda().eval()
    
    # Initialize the classification accuracy (clean & attack)
    clean_vgg16, clean_vgg19, clean_res50, clean_res152, clean_dense121, clean_dense169 = 0,0,0,0,0,0
    acc_vgg16, acc_vgg19, acc_res50, acc_res152, acc_dense121, acc_dense169 = 0,0,0,0,0,0
    clean, accuracy = 0, 0
    
    # Evaluation loop
    for i, data_val in tqdm(enumerate(ds_val)):
        img, label = data_val
        img = Variable(torch.FloatTensor(img)).cuda()
        label = Variable(torch.from_numpy(np.array(label)).long().cuda())
        adv = netG(img)

        # projection
        adv = torch.min(torch.max(adv, img - args.eps/255.0), img + args.eps/255.0)
        adv = torch.clamp(adv, 0.0, 1.0)
        
        with torch.no_grad():
            if domain == 'imagenet':
                clean_vgg16 += torch.sum(torch.argmax(model_vgg16(img), dim=1) == label.cuda())
                acc_vgg16 += torch.sum(torch.argmax(model_vgg16(adv), dim=1) == label.cuda())
                
                clean_vgg19 += torch.sum(torch.argmax(model_vgg19(img), dim=1) == label.cuda())
                acc_vgg19 += torch.sum(torch.argmax(model_vgg19(adv), dim=1) == label.cuda())
                
                clean_res50 += torch.sum(torch.argmax(model_res50(img), dim=1) == label.cuda())
                acc_res50 += torch.sum(torch.argmax(model_res50(adv), dim=1) == label.cuda())
                
                clean_res152 += torch.sum(torch.argmax(model_res152(img), dim=1) == label.cuda())
                acc_res152 += torch.sum(torch.argmax(model_res152(adv), dim=1) == label.cuda())
                
                clean_dense121 += torch.sum(torch.argmax(model_dense121(img), dim=1) == label.cuda())
                acc_dense121 += torch.sum(torch.argmax(model_dense121(adv), dim=1) == label.cuda())
                
                clean_dense169 += torch.sum(torch.argmax(model_dense169(img), dim=1) == label.cuda())
                acc_dense169 += torch.sum(torch.argmax(model_dense169(adv), dim=1) == label.cuda())
            
            elif domain[:3] != 'dcl': # CIFAR-10, CIFAR-100, STL-10, SVHN
                clean += torch.sum(torch.argmax(model_raw(img), dim=1) == label.cuda())
                accuracy += torch.sum(torch.argmax(model_raw(adv), dim=1) == label.cuda())
                
            else: # CUB-200-2011, Stanford Cars, FGVC Aircraft
                outputs = model_res50(adv)
                outputs_clean = model_res50(img)
                outputs_pred = outputs[0] + outputs[1][:,0:numcls] + outputs[1][:,numcls:2*numcls]
                outputs_pred_clean = outputs_clean[0] + outputs_clean[1][:,0:numcls] + outputs_clean[1][:,numcls:2*numcls]
                acc_res50 += torch.sum(torch.argmax(outputs_pred, dim=1) == label.cuda())
                clean_res50 += torch.sum(torch.argmax(outputs_pred_clean, dim=1) == label.cuda())
                
                outputs2 = model_senet(adv)
                outputs_clean2 = model_senet(img)
                outputs_pred2 = outputs2[0] + outputs2[1][:,0:numcls] + outputs2[1][:,numcls:2*numcls]
                outputs_pred_clean2 = outputs_clean2[0] + outputs_clean2[1][:,0:numcls] + outputs_clean2[1][:,numcls:2*numcls]
                acc_senet += torch.sum(torch.argmax(outputs_pred2, dim=1) == label.cuda())
                clean_senet += torch.sum(torch.argmax(outputs_pred_clean2, dim=1) == label.cuda())
                
                outputs3 = model_seres101(adv)
                outputs_clean3 = model_seres101(img)
                outputs_pred3 = outputs3[0] + outputs3[1][:,0:numcls] + outputs3[1][:,numcls:2*numcls]
                outputs_pred_clean3 = outputs_clean3[0] + outputs_clean3[1][:,0:numcls] + outputs_clean3[1][:,numcls:2*numcls]
                acc_seres101 += torch.sum(torch.argmax(outputs_pred3, dim=1) == label.cuda())
                clean_seres101 += torch.sum(torch.argmax(outputs_pred_clean3, dim=1) == label.cuda())
        
    if domain == 'imagenet':
        print('----------------vgg16----------------')
        print(clean_vgg16 / data_length)
        print(acc_vgg16 / data_length)
        print('----------------vgg19----------------')
        print(clean_vgg19 / data_length)
        print(acc_vgg19 / data_length)
        print('----------------res50----------------')
        print(clean_res50 / data_length)
        print(acc_res50 / data_length)      
        print('----------------res152----------------')
        print(clean_res152 / data_length)
        print(acc_res152 / data_length)
        print('----------------dense121----------------')
        print(clean_dense121 / data_length)
        print(acc_dense121 / data_length)
        print('----------------dense169----------------')
        print(clean_dense169 / data_length)
        print(acc_dense169 / data_length)
        
    elif domain[:3] == 'dcl': # CUB-200-2011, Stanford Cars, FGVC Aircraft
        print('----------------backbone:res50----------------')
        print(clean_res50 / data_length)
        print(acc_res50 / data_length)
        print('----------------backbone:se-net----------------')
        print(clean_senet / data_length)
        print(acc_senet / data_length)
        print('----------------backbone:se-res101----------------')
        print(clean_seres101 / data_length)
        print(acc_seres101 / data_length)

    else: # CIFAR-10, CIFAR-100, STL-10, SVHN
        print(clean / data_length)
        print(accuracy / data_length)


============================== dcl_car ==============================
resnet50
senet154


Downloading: "http://data.lip6.fr/cadene/pretrainedmodels/senet154-c7b49a05.pth" to /home/vilab/.cache/torch/hub/checkpoints/senet154-c7b49a05.pth


  0%|          | 0.00/440M [00:00<?, ?B/s]

se_resnet101


Downloading: "http://data.lip6.fr/cadene/pretrainedmodels/se_resnet101-7e38fcc6.pth" to /home/vilab/.cache/torch/hub/checkpoints/se_resnet101-7e38fcc6.pth


  0%|          | 0.00/189M [00:00<?, ?B/s]

Validation data length: 8041
Substitute Model: vgg16 	 RN: False 	 DA: False 	 FA: False 	 Saving instance: 0


0it [00:00, ?it/s]


FileNotFoundError: Caught FileNotFoundError in DataLoader worker process 0.
Original Traceback (most recent call last):
  File "/home/vilab/anaconda3/envs/bia/lib/python3.7/site-packages/torch/utils/data/_utils/worker.py", line 202, in _worker_loop
    data = fetcher.fetch(index)
  File "/home/vilab/anaconda3/envs/bia/lib/python3.7/site-packages/torch/utils/data/_utils/fetch.py", line 44, in fetch
    data = [self.dataset[idx] for idx in possibly_batched_index]
  File "/home/vilab/anaconda3/envs/bia/lib/python3.7/site-packages/torch/utils/data/_utils/fetch.py", line 44, in <listcomp>
    data = [self.dataset[idx] for idx in possibly_batched_index]
  File "/home/vilab/yhm/Beyond-ImageNet-Attack/DCL_finegrained/dataset_DCL.py", line 73, in __getitem__
    img = self.pil_loader(img_path)
  File "/home/vilab/yhm/Beyond-ImageNet-Attack/DCL_finegrained/dataset_DCL.py", line 112, in pil_loader
    with open(imgpath, 'rb') as f:
FileNotFoundError: [Errno 2] No such file or directory: '/home/vi

##### GT
netG_BIA_9.pth
============================== cifar10 ==============================
Building and initializing cifar10 parameters
Building CIFAR-10 data loader with 1 workers
Files already downloaded and verified
Building CIFAR-10 data loader with 1 workers
Files already downloaded and verified
Validation data length: 10000
Substitute Model: vgg16 	 RN: False 	 DA: False 	 FA: False 	 Saving instance: 9
79it [00:26,  2.96it/s]
tensor(0.9378, device='cuda:0')
tensor(0.5649, device='cuda:0')
============================== cifar100 ==============================
Building and initializing cifar100 parameters
Building CIFAR-100 data loader with 1 workers
Files already downloaded and verified
Building CIFAR-100 data loader with 1 workers
Files already downloaded and verified
Validation data length: 10000
Substitute Model: vgg16 	 RN: False 	 DA: False 	 FA: False 	 Saving instance: 9
79it [00:03, 21.14it/s]
tensor(0.7427, device='cuda:0')
tensor(0.2203, device='cuda:0')
============================== stl10 ==============================
Building and initializing stl10 parameters
Building STL10 data loader with 1 workers
Files already downloaded and verified
Building STL10 data loader with 1 workers
Files already downloaded and verified
Validation data length: 8000
Substitute Model: vgg16 	 RN: False 	 DA: False 	 FA: False 	 Saving instance: 9
63it [00:03, 18.60it/s]
tensor(0.7759, device='cuda:0')
tensor(0.7005, device='cuda:0')
============================== svhn ==============================
Building and initializing svhn parameters
Building SVHN data loader with 1 workers
Using downloaded and verified file: ./public_dataset/pytorch/svhn-data/test_32x32.mat
Building SVHN data loader with 1 workers
Using downloaded and verified file: ./public_dataset/pytorch/svhn-data/test_32x32.mat
Validation data length: 26032
Substitute Model: vgg16 	 RN: False 	 DA: False 	 FA: False 	 Saving instance: 9
204it [00:06, 32.07it/s]
tensor(0.9603, device='cuda:0')
tensor(0.9046, device='cuda:0')
============================== imagenet ==============================
Building and initializing imagenet parameters
Building IMAGENET data loader, 50000 for train, 50000 for test
Building IMAGENET data loader, 50000 for train, 50000 for test
Validation data length: 50000
Substitute Model: vgg16 	 RN: False 	 DA: False 	 FA: False 	 Saving instance: 9
1563it [14:46,  1.76it/s]
----------------vgg16----------------
tensor(0.7014, device='cuda:0')
tensor(0.0133, device='cuda:0')
----------------vgg19----------------
tensor(0.7095, device='cuda:0')
tensor(0.0310, device='cuda:0')
----------------res50----------------
tensor(0.7460, device='cuda:0')
tensor(0.2396, device='cuda:0')
----------------res152----------------
tensor(0.7734, device='cuda:0')
tensor(0.4146, device='cuda:0')
----------------dense121----------------
tensor(0.7422, device='cuda:0')
tensor(0.2464, device='cuda:0')
----------------dense169----------------
tensor(0.7575, device='cuda:0')
tensor(0.3069, device='cuda:0')

##### Iterative training : 1 normal batch , 1 FA batch, 1 normal batch, ...
netG_BIA+FA_0.pth   # 1epoch

============================== cifar10 ==============================
Building and initializing cifar10 parameters
Building CIFAR-10 data loader with 1 workers
Files already downloaded and verified
Building CIFAR-10 data loader with 1 workers
Files already downloaded and verified
Validation data length: 10000
Substitute Model: vgg16 	 RN: False 	 DA: False 	 FA: True 	 Saving instance: 0
79it [00:28,  2.79it/s]
tensor(0.9378, device='cuda:0')
tensor(0.5676, device='cuda:0')
============================== cifar100 ==============================
Building and initializing cifar100 parameters
Building CIFAR-100 data loader with 1 workers
Files already downloaded and verified
Building CIFAR-100 data loader with 1 workers
Files already downloaded and verified
Validation data length: 10000
Substitute Model: vgg16 	 RN: False 	 DA: False 	 FA: True 	 Saving instance: 0
79it [00:03, 21.71it/s]
tensor(0.7427, device='cuda:0')
tensor(0.2350, device='cuda:0')
============================== stl10 ==============================
Building and initializing stl10 parameters
Building STL10 data loader with 1 workers
Files already downloaded and verified
Building STL10 data loader with 1 workers
Files already downloaded and verified
Validation data length: 8000
Substitute Model: vgg16 	 RN: False 	 DA: False 	 FA: True 	 Saving instance: 0
63it [00:03, 17.23it/s]
tensor(0.7759, device='cuda:0')
tensor(0.6859, device='cuda:0')
============================== svhn ==============================
Building and initializing svhn parameters
Building SVHN data loader with 1 workers
Using downloaded and verified file: ./public_dataset/pytorch/svhn-data/test_32x32.mat
Building SVHN data loader with 1 workers
Using downloaded and verified file: ./public_dataset/pytorch/svhn-data/test_32x32.mat
Validation data length: 26032
Substitute Model: vgg16 	 RN: False 	 DA: False 	 FA: True 	 Saving instance: 0
204it [00:06, 31.70it/s]
tensor(0.9603, device='cuda:0')
tensor(0.8544, device='cuda:0')
============================== imagenet ==============================
Building and initializing imagenet parameters
Building IMAGENET data loader, 50000 for train, 50000 for test
Building IMAGENET data loader, 50000 for train, 50000 for test
Validation data length: 50000
Substitute Model: vgg16 	 RN: False 	 DA: False 	 FA: True 	 Saving instance: 0
143it [01:41,  1.72it/s]
----------------vgg16----------------
tensor(0.7014, device='cuda:0')
tensor(0.0264, device='cuda:0')
----------------vgg19----------------
tensor(0.7095, device='cuda:0')
tensor(0.0505, device='cuda:0')
----------------res50----------------
tensor(0.7460, device='cuda:0')
tensor(0.2480, device='cuda:0')
----------------res152----------------
tensor(0.7734, device='cuda:0')
tensor(0.3985, device='cuda:0')
----------------dense121----------------
tensor(0.7422, device='cuda:0')
tensor(0.2855, device='cuda:0')
----------------dense169----------------
tensor(0.7575, device='cuda:0')
tensor(0.3547, device='cuda:0')

netG_BIA+FA_0_10000.pth     # 1 epoch + 10000 iterations 

============================== cifar10 ==============================
Building and initializing cifar10 parameters
Building CIFAR-10 data loader with 1 workers
Files already downloaded and verified
Building CIFAR-10 data loader with 1 workers
Files already downloaded and verified
Validation data length: 10000
Substitute Model: vgg16 	 RN: False 	 DA: False 	 FA: True 	 Saving instance: 0
79it [00:03, 21.03it/s]
tensor(0.9378, device='cuda:0')
tensor(0.5524, device='cuda:0')
============================== cifar100 ==============================
Building and initializing cifar100 parameters
Building CIFAR-100 data loader with 1 workers
Files already downloaded and verified
Building CIFAR-100 data loader with 1 workers
Files already downloaded and verified
Validation data length: 10000
Substitute Model: vgg16 	 RN: False 	 DA: False 	 FA: True 	 Saving instance: 0
79it [00:03, 20.52it/s]
tensor(0.7427, device='cuda:0')
tensor(0.2303, device='cuda:0')
============================== stl10 ==============================
Building and initializing stl10 parameters
Building STL10 data loader with 1 workers
Files already downloaded and verified
Building STL10 data loader with 1 workers
Files already downloaded and verified
Validation data length: 8000
Substitute Model: vgg16 	 RN: False 	 DA: False 	 FA: True 	 Saving instance: 0
63it [00:03, 20.02it/s]
tensor(0.7759, device='cuda:0')
tensor(0.6795, device='cuda:0')
============================== svhn ==============================
Building and initializing svhn parameters
Building SVHN data loader with 1 workers
Using downloaded and verified file: ./public_dataset/pytorch/svhn-data/test_32x32.mat
Building SVHN data loader with 1 workers
Using downloaded and verified file: ./public_dataset/pytorch/svhn-data/test_32x32.mat
Validation data length: 26032
Substitute Model: vgg16 	 RN: False 	 DA: False 	 FA: True 	 Saving instance: 0
204it [00:06, 30.75it/s]
tensor(0.9603, device='cuda:0')
tensor(0.8762, device='cuda:0')
============================== imagenet ==============================
Building and initializing imagenet parameters
Building IMAGENET data loader, 50000 for train, 50000 for test
1563it [14:44,  1.77it/s]
----------------vgg16----------------
tensor(0.7014, device='cuda:0')
tensor(0.0245, device='cuda:0')
----------------vgg19----------------
tensor(0.7095, device='cuda:0')
tensor(0.0479, device='cuda:0')
----------------res50----------------
tensor(0.7460, device='cuda:0')
tensor(0.2529, device='cuda:0')
----------------res152----------------
tensor(0.7734, device='cuda:0')
tensor(0.4097, device='cuda:0')
----------------dense121----------------
tensor(0.7422, device='cuda:0')
tensor(0.2871, device='cuda:0')
----------------dense169----------------
tensor(0.7575, device='cuda:0')
tensor(0.3600, device='cuda:0')